In [1]:
from ipywidgets import VBox, HBox
from pyspark.sql import SparkSession
from perspective import PerspectiveWidget

from helpers.data import (
    HOST,
    MACHINES_PORT,
    USAGE_PORT,
    STATUS_PORT,
    JOBS_PORT,
)
from helpers.spark import (
    MACHINE_SCHEMA,
    MACHINE_SCHEMA_SPARK,
    USAGE_SCHEMA,
    USAGE_SCHEMA_SPARK,
    STATUS_SCHEMA,
    STATUS_SCHEMA_SPARK,
    JOBS_SCHEMA,
    JOBS_SCHEMA_SPARK,
)

In [2]:
# Important imports
from helpers.spark import (
    get_df_from_server,
    push_to_perspective,
)
from helpers.fastapi import (
    perspective_spark_bridge,
    start_server,
)

In [3]:
spark = SparkSession.builder.appName("Perspective Demo").getOrCreate()

24/06/07 16:18:16 WARN Utils: Your hostname, mbp-m1 resolves to a loopback address: 127.0.0.1; using 10.0.1.10 instead (on interface en0)
24/06/07 16:18:16 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/07 16:18:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
# Get spark streaming dfs
machines_df = get_df_from_server(spark, MACHINE_SCHEMA_SPARK, HOST, MACHINES_PORT)
usage_df = get_df_from_server(spark, USAGE_SCHEMA_SPARK, HOST, USAGE_PORT)
status_df = get_df_from_server(spark, STATUS_SCHEMA_SPARK, HOST, STATUS_PORT)
jobs_df = get_df_from_server(spark, JOBS_SCHEMA_SPARK, HOST, JOBS_PORT)

24/06/07 16:18:17 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.
24/06/07 16:18:17 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.
24/06/07 16:18:17 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.
24/06/07 16:18:17 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.


In [5]:
# construct 4 separate perspective widgets. Each will have its own table internally
machines_widget = PerspectiveWidget(MACHINE_SCHEMA, index="machine_id", settings=False)
usage_widget = PerspectiveWidget(USAGE_SCHEMA, index="machine_id", settings=False)
status_widget = PerspectiveWidget(STATUS_SCHEMA, index="machine_id", sort=[["last_update", "desc"]], settings=False)
jobs_widget = PerspectiveWidget(JOBS_SCHEMA, sort=[["start_time", "desc"]], settings=False)

In [6]:
# a little bit of layout with ipywidgets
VBox(children=[
    HBox(children=[machines_widget, usage_widget]),
    HBox(children=[status_widget, jobs_widget]),
])

In [7]:
# %%capture
app = perspective_spark_bridge(
    {
        "machines": machines_widget,
        "usage": usage_widget,
        "status": status_widget,
        "jobs": jobs_widget,
    }
)
port = start_server(app)

CRITICAL:root:Listening on http://localhost:55248
INFO:     Started server process [6117]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:55248 (Press CTRL+C to quit)
INFO:     ('127.0.0.1', 55270) - "WebSocket /tables/status" [accepted]
INFO:     connection open
INFO:     ('127.0.0.1', 55272) - "WebSocket /tables/machines" [accepted]
INFO:     ('127.0.0.1', 55275) - "WebSocket /tables/jobs" [accepted]
INFO:     connection open
INFO:     connection open
INFO:     ('127.0.0.1', 55276) - "WebSocket /tables/usage" [accepted]
INFO:     connection open
INFO:     ('127.0.0.1', 55290) - "WebSocket /tables/machines" [accepted]
INFO:     connection open
INFO:     ('127.0.0.1', 55292) - "WebSocket /tables/usage" [accepted]
INFO:     connection open
INFO:     ('127.0.0.1', 55294) - "WebSocket /tables/jobs" [accepted]
INFO:     connection open
INFO:     ('127.0.0.1', 55296) - "WebSocket /tables/status" [accepted]
INFO: 

INFO:     connection open
INFO:     connection closed
INFO:     ('127.0.0.1', 55571) - "WebSocket /tables/status" [accepted]
INFO:     connection open
INFO:     ('127.0.0.1', 55573) - "WebSocket /tables/jobs" [accepted]
INFO:     connection open
INFO:     ('127.0.0.1', 55575) - "WebSocket /tables/usage" [accepted]
INFO:     connection open
INFO:     ('127.0.0.1', 55583) - "WebSocket /tables/status" [accepted]
INFO:     connection open
INFO:     ('127.0.0.1', 55585) - "WebSocket /tables/jobs" [accepted]
INFO:     connection open
INFO:     ('127.0.0.1', 55587) - "WebSocket /tables/usage" [accepted]
INFO:     connection open
INFO:     ('127.0.0.1', 55595) - "WebSocket /tables/status" [accepted]
INFO:     connection open
INFO:     ('127.0.0.1', 55597) - "WebSocket /tables/jobs" [accepted]
INFO:     connection open
INFO:     ('127.0.0.1', 55599) - "WebSocket /tables/usage" [accepted]
INFO:     connection open
INFO:     ('127.0.0.1', 55607) - "WebSocket /tables/jobs" [accepted]
INFO:     con

INFO:     connection closed
INFO:     connection closed
INFO:     connection closed
INFO:     connection closed
INFO:     connection closed
INFO:     connection closed
INFO:     connection closed
INFO:     ('127.0.0.1', 55844) - "WebSocket /tables/jobs" [accepted]
INFO:     ('127.0.0.1', 55845) - "WebSocket /tables/status" [accepted]
INFO:     connection open
INFO:     connection open
INFO:     ('127.0.0.1', 55847) - "WebSocket /tables/usage" [accepted]
INFO:     connection open
INFO:     ('127.0.0.1', 55859) - "WebSocket /tables/jobs" [accepted]
INFO:     connection open
INFO:     ('127.0.0.1', 55861) - "WebSocket /tables/status" [accepted]
INFO:     connection open
INFO:     ('127.0.0.1', 55863) - "WebSocket /tables/usage" [accepted]
INFO:     connection open
INFO:     connection closed
INFO:     connection closed
INFO:     connection closed
INFO:     ('127.0.0.1', 55872) - "WebSocket /tables/jobs" [accepted]
INFO:     connection open
INFO:     ('127.0.0.1', 55873) - "WebSocket /tabl

INFO:     connection open
INFO:     connection open
INFO:     connection closed
INFO:     connection closed
INFO:     ('127.0.0.1', 56139) - "WebSocket /tables/usage" [accepted]
INFO:     connection open
INFO:     ('127.0.0.1', 56147) - "WebSocket /tables/status" [accepted]
INFO:     connection open
INFO:     ('127.0.0.1', 56149) - "WebSocket /tables/jobs" [accepted]
INFO:     connection open
INFO:     ('127.0.0.1', 56151) - "WebSocket /tables/usage" [accepted]
INFO:     connection open
INFO:     connection closed
INFO:     ('127.0.0.1', 56161) - "WebSocket /tables/status" [accepted]
INFO:     connection open
INFO:     ('127.0.0.1', 56163) - "WebSocket /tables/jobs" [accepted]
INFO:     connection open
INFO:     ('127.0.0.1', 56165) - "WebSocket /tables/usage" [accepted]
INFO:     connection open
INFO:     ('127.0.0.1', 56174) - "WebSocket /tables/status" [accepted]
INFO:     ('127.0.0.1', 56175) - "WebSocket /tables/jobs" [accepted]
INFO:     connection open
INFO:     connection open


INFO:     connection open
INFO:     ('127.0.0.1', 56410) - "WebSocket /tables/status" [accepted]
INFO:     connection open
INFO:     ('127.0.0.1', 56413) - "WebSocket /tables/jobs" [accepted]
INFO:     ('127.0.0.1', 56414) - "WebSocket /tables/usage" [accepted]
INFO:     connection open
INFO:     connection open
INFO:     connection closed
INFO:     ('127.0.0.1', 56422) - "WebSocket /tables/status" [accepted]
INFO:     connection open
INFO:     ('127.0.0.1', 56425) - "WebSocket /tables/jobs" [accepted]
INFO:     ('127.0.0.1', 56426) - "WebSocket /tables/usage" [accepted]
INFO:     connection open
INFO:     connection open
INFO:     ('127.0.0.1', 56439) - "WebSocket /tables/status" [accepted]
INFO:     ('127.0.0.1', 56440) - "WebSocket /tables/jobs" [accepted]
INFO:     connection open
INFO:     connection open
INFO:     connection closed
INFO:     connection closed
INFO:     connection closed
INFO:     ('127.0.0.1', 56442) - "WebSocket /tables/usage" [accepted]
INFO:     connection ope

...

INFO:     ('127.0.0.1', 63736) - "WebSocket /tables/jobs" [accepted]
INFO:     connection open
INFO:     ('127.0.0.1', 63738) - "WebSocket /tables/status" [accepted]
INFO:     connection open
INFO:     ('127.0.0.1', 63740) - "WebSocket /tables/usage" [accepted]
INFO:     connection open
INFO:     connection closed
INFO:     connection closed
INFO:     connection closed
INFO:     connection closed
INFO:     connection closed
INFO:     connection closed
INFO:     connection closed
INFO:     connection closed
INFO:     connection closed
INFO:     connection closed
INFO:     connection closed
INFO:     connection closed
INFO:     connection closed
INFO:     connection closed
INFO:     connection closed
INFO:     connection closed
INFO:     connection closed
INFO:     connection closed
INFO:     connection closed
INFO:     connection closed
INFO:     connection closed
INFO:     connection closed
INFO:     connection closed
INFO:     connection closed
INFO:     connection closed
INFO:     co

INFO:     connection closed
INFO:     connection closed
INFO:     ('127.0.0.1', 63990) - "WebSocket /tables/status" [accepted]
INFO:     ('127.0.0.1', 63992) - "WebSocket /tables/jobs" [accepted]
INFO:     connection open
INFO:     connection open
INFO:     ('127.0.0.1', 63994) - "WebSocket /tables/usage" [accepted]
INFO:     connection open
INFO:     ('127.0.0.1', 64004) - "WebSocket /tables/status" [accepted]
INFO:     ('127.0.0.1', 64003) - "WebSocket /tables/jobs" [accepted]
INFO:     connection open
INFO:     connection open
INFO:     ('127.0.0.1', 64006) - "WebSocket /tables/usage" [accepted]
INFO:     connection open
INFO:     ('127.0.0.1', 64025) - "WebSocket /tables/status" [accepted]
INFO:     connection open
INFO:     ('127.0.0.1', 64027) - "WebSocket /tables/jobs" [accepted]
INFO:     connection open
INFO:     ('127.0.0.1', 64029) - "WebSocket /tables/usage" [accepted]
INFO:     connection open
INFO:     ('127.0.0.1', 64037) - "WebSocket /tables/jobs" [accepted]
INFO:     c

INFO:     ('127.0.0.1', 64198) - "WebSocket /tables/jobs" [accepted]
INFO:     ('127.0.0.1', 64200) - "WebSocket /tables/usage" [accepted]
INFO:     connection open
INFO:     connection open
INFO:     connection open
INFO:     ('127.0.0.1', 64208) - "WebSocket /tables/status" [accepted]
INFO:     ('127.0.0.1', 64210) - "WebSocket /tables/jobs" [accepted]
INFO:     ('127.0.0.1', 64212) - "WebSocket /tables/usage" [accepted]
INFO:     connection open
INFO:     connection open
INFO:     connection open
INFO:     ('127.0.0.1', 64221) - "WebSocket /tables/status" [accepted]
INFO:     ('127.0.0.1', 64222) - "WebSocket /tables/jobs" [accepted]
INFO:     ('127.0.0.1', 64224) - "WebSocket /tables/usage" [accepted]
INFO:     connection open
INFO:     connection open
INFO:     connection open
INFO:     ('127.0.0.1', 64233) - "WebSocket /tables/usage" [accepted]
INFO:     ('127.0.0.1', 64234) - "WebSocket /tables/jobs" [accepted]
INFO:     ('127.0.0.1', 64236) - "WebSocket /tables/status" [accepte

INFO:     ('127.0.0.1', 64500) - "WebSocket /tables/usage" [accepted]
INFO:     connection open
INFO:     connection open
INFO:     connection open
INFO:     ('127.0.0.1', 64508) - "WebSocket /tables/jobs" [accepted]
INFO:     ('127.0.0.1', 64510) - "WebSocket /tables/status" [accepted]
INFO:     connection open
INFO:     connection open
ERROR:    closing handshake failed
Traceback (most recent call last):
  File "/opt/homebrew/lib/python3.11/site-packages/websockets/legacy/protocol.py", line 968, in transfer_data
    message = await self.read_message()
              ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/lib/python3.11/site-packages/websockets/legacy/protocol.py", line 1038, in read_message
    frame = await self.read_data_frame(max_size=self.max_size)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/lib/python3.11/site-packages/websockets/legacy/protocol.py", line 1113, in read_data_frame
    frame = await self.read_frame(max_size)
       

INFO:     connection open
INFO:     connection open
INFO:     connection open
INFO:     ('127.0.0.1', 64697) - "WebSocket /tables/status" [accepted]
INFO:     ('127.0.0.1', 64699) - "WebSocket /tables/jobs" [accepted]
INFO:     connection open
INFO:     connection open
INFO:     ('127.0.0.1', 64701) - "WebSocket /tables/usage" [accepted]
INFO:     connection open
INFO:     ('127.0.0.1', 64710) - "WebSocket /tables/status" [accepted]
INFO:     connection open
INFO:     ('127.0.0.1', 64712) - "WebSocket /tables/usage" [accepted]
INFO:     connection open
INFO:     ('127.0.0.1', 64714) - "WebSocket /tables/jobs" [accepted]
INFO:     connection open
INFO:     ('127.0.0.1', 64723) - "WebSocket /tables/jobs" [accepted]
INFO:     connection open
INFO:     ('127.0.0.1', 64726) - "WebSocket /tables/status" [accepted]
INFO:     ('127.0.0.1', 64727) - "WebSocket /tables/usage" [accepted]
INFO:     connection open
INFO:     connection open
INFO:     ('127.0.0.1', 64736) - "WebSocket /tables/jobs" 

In [8]:
push_to_perspective(machines_df, "machines", "localhost", port)
push_to_perspective(usage_df, "usage", "localhost", port)
push_to_perspective(status_df, "status", "localhost", port)
push_to_perspective(jobs_df, "jobs", "localhost", port)

24/06/07 16:18:23 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/pm/0nt85wh51h39f604mgmyyfwm0000gn/T/temporary-e76a92fb-2517-4b9c-89cf-012a443a7902. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/06/07 16:18:23 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
24/06/07 16:18:23 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/pm/0nt85wh51h39f604mgmyyfwm0000gn/T/temporary-63da1257-1e1c-4e6b-86a9-e409ae532f46. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folde

In [9]:
status_widget.plugin = "X Bar"
status_widget.group_by = ["status"]
status_widget.columns = ["machine_id"]
status_widget.aggregates = {"status": "last"}

In [11]:
jobs_widget.group_by = ["machine_id"]
jobs_widget.columns = ["job_id", "name", "units", "start_time", "end_time"],
jobs_widget.aggregates = {"job_id": "count", "name": "last", "units": "sum", "start_time": "last", "end_time": "last"}